In [1]:
import select_sensor
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import traceback
import pandas as pd
from select_sensor import SelectSensor
import seaborn as sns
import math

In [2]:
select_sensor = SelectSensor('config.json')

select_sensor.read_init_sensor('sensor.txt')
select_sensor.read_mean_std('mean_std.txt')
select_sensor.compute_multivariant_gaussian()
print('error ', select_sensor.test_error())
select_sensor.print()

Computed covariance!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
error  0.15512
(0, 0)  8
(0, 1)  8
(0, 2)  8
(0, 3)  8
(0, 4)  7
(0, 5)  6
(0, 6)  1
(0, 7)  2
(0, 8)  0
(0, 9)  0
(0, 10)  2
(0, 11)  2
(0, 12)  2
(0, 13)  1
(0, 14)  3
(0, 15)  0
(0, 16)  1
(0, 17)  1
(0, 18)  0
(0, 19)  3
(0, 20)  3
(0, 21)  5
(0, 22)  3
(0, 23)  5
(0, 24)  3
(0, 25)  0
(0, 26)  0
(0, 27)  1
(0, 28)  0
(0, 29)  2
(0, 30)  0
(0, 31)  0
(0, 32)  0
(0, 33)  2
(0, 34)  0
(0, 35)  1
(0, 36)  4
(0, 37)  6
(0, 38)  9
(0, 39)  8
(0, 40)  7
(0, 41)  4
(0, 42)  4
(0, 43)  6
(0, 44)  3
(0, 45)  2
(0, 46)  0
(0, 47)  0
(0, 48)  0
(0, 49)  0
(1, 0)  7
(1, 1)  8
(1, 2)  7
(1, 3)  9
(1, 4)  9
(1, 5)  3
(1, 6)  4
(1, 7)  0
(1, 8)  0
(1, 9)  0
(1, 10)  0
(1, 11)  0
(1, 12)  0
(1, 13)  1
(1, 14)  0
(1, 15)  0
(1, 16)  0
(1, 17)  0
(1, 18)  0
(1, 19)  0
(1, 20)  3
(1, 21)  4
(1, 22)  3
(1, 23)  4
(1, 24)  2
(1, 25)  1
(1, 26) 

(17, 18)  0
(17, 19)  2
(17, 20)  0
(17, 21)  0
(17, 22)  0
(17, 23)  1
(17, 24)  0
(17, 25)  1
(17, 26)  0
(17, 27)  2
(17, 28)  5
(17, 29)  0
(17, 30)  0
(17, 31)  0
(17, 32)  0
(17, 33)  0
(17, 34)  2
(17, 35)  5
(17, 36)  5
(17, 37)  1
(17, 38)  1
(17, 39)  1
(17, 40)  0
(17, 41)  0
(17, 42)  0
(17, 43)  1
(17, 44)  0
(17, 45)  0
(17, 46)  0
(17, 47)  0
(17, 48)  2
(17, 49)  4
(18, 0)  3
(18, 1)  3
(18, 2)  7
(18, 3)  4
(18, 4)  5
(18, 5)  6
(18, 6)  6
(18, 7)  2
(18, 8)  2
(18, 9)  2
(18, 10)  0
(18, 11)  2
(18, 12)  0
(18, 13)  0
(18, 14)  0
(18, 15)  1
(18, 16)  2
(18, 17)  3
(18, 18)  2
(18, 19)  0
(18, 20)  0
(18, 21)  0
(18, 22)  1
(18, 23)  2
(18, 24)  0
(18, 25)  0
(18, 26)  3
(18, 27)  1
(18, 28)  3
(18, 29)  0
(18, 30)  0
(18, 31)  0
(18, 32)  1
(18, 33)  0
(18, 34)  3
(18, 35)  6
(18, 36)  6
(18, 37)  6
(18, 38)  2
(18, 39)  1
(18, 40)  0
(18, 41)  2
(18, 42)  2
(18, 43)  1
(18, 44)  1
(18, 45)  1
(18, 46)  0
(18, 47)  2
(18, 48)  2
(18, 49)  3
(19, 0)  2
(19, 1)  7
(19,

(45, 44)  0
(45, 45)  0
(45, 46)  0
(45, 47)  0
(45, 48)  0
(45, 49)  2
(46, 0)  1
(46, 1)  0
(46, 2)  0
(46, 3)  0
(46, 4)  0
(46, 5)  0
(46, 6)  0
(46, 7)  0
(46, 8)  0
(46, 9)  0
(46, 10)  0
(46, 11)  1
(46, 12)  0
(46, 13)  3
(46, 14)  2
(46, 15)  5
(46, 16)  8
(46, 17)  5
(46, 18)  5
(46, 19)  5
(46, 20)  5
(46, 21)  6
(46, 22)  7
(46, 23)  3
(46, 24)  1
(46, 25)  2
(46, 26)  0
(46, 27)  0
(46, 28)  0
(46, 29)  0
(46, 30)  1
(46, 31)  0
(46, 32)  0
(46, 33)  0
(46, 34)  0
(46, 35)  0
(46, 36)  0
(46, 37)  0
(46, 38)  0
(46, 39)  0
(46, 40)  0
(46, 41)  0
(46, 42)  0
(46, 43)  0
(46, 44)  0
(46, 45)  0
(46, 46)  0
(46, 47)  0
(46, 48)  1
(46, 49)  4
(47, 0)  0
(47, 1)  0
(47, 2)  0
(47, 3)  1
(47, 4)  0
(47, 5)  0
(47, 6)  0
(47, 7)  0
(47, 8)  0
(47, 9)  0
(47, 10)  0
(47, 11)  0
(47, 12)  0
(47, 13)  2
(47, 14)  7
(47, 15)  4
(47, 16)  5
(47, 17)  6
(47, 18)  6
(47, 19)  9
(47, 20)  7
(47, 21)  4
(47, 22)  7
(47, 23)  3
(47, 24)  6
(47, 25)  5
(47, 26)  0
(47, 27)  2
(47, 28)  0


In [ ]:
x = np.linspace(-10, 10, 100)
y = multivariate_normal.pdf(x, mean=0, cov=1); y

plt.plot(x, y)
plt.show()

In [ ]:
x, y = np.mgrid[0:50:1, 0:50:1]
pos = np.zeros(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y
rv = multivariate_normal([25, 25], [[400, 0], [0, 400]])
plt.figure(figsize=(7,7))
plt.contourf(x, y, rv.pdf(pos))
plt.show()

In [ ]:
rv.pdf(pos).max()